In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
import xgboost as xgb
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

os.chdir("/content/drive/MyDrive/ESAA/프로젝트2_병원개폐업_분류예측/9565_hospital_data/")

Mounted at /content/drive


In [51]:
#Reading the train and test files
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 변수 뽑기

In [53]:
train = train[['OC','revenue1','profit1','receivableS1','debt1','longLoan1','surplus1','employee1','employee2']]
train.head()

,OC,revenue1,profit1,receivableS1,debt1,longLoan1,surplus1,employee1,employee2
0,open,4.217530e+09,225169678.0,4.700557e+08,6.828260e+08,3.510000e+08,1.496394e+09,62.0,64.0
1,open,NaN,NaN,NaN,NaN,NaN,NaN,801.0,813.0
2,open,1.004522e+09,41864754.0,8.095950e+06,9.241434e+07,0.000000e+00,0.000000e+00,234.0,1.0
3,open,7.250734e+10,902830288.0,0.000000e+00,4.236380e+10,7.253040e+09,8.643659e+09,663.0,663.0
4,open,4.904354e+10,80749696.0,4.099320e+09,4.989811e+10,1.350000e+10,9.025550e+09,206.0,197.0


In [54]:
test = test[['revenue1','profit1','receivableS1','debt1','longLoan1','surplus1','employee1','employee2']]
test.head()

,revenue1,profit1,receivableS1,debt1,longLoan1,surplus1,employee1,employee2
0,6.682486e+10,5.036692e+08,7.242580e+08,6.769090e+10,6.319097e+09,8.944587e+09,693,693
1,3.495758e+10,4.525557e+08,6.610290e+08,6.626737e+10,1.720000e+10,1.046277e+10,379,371
2,2.326031e+10,-2.148749e+09,7.337107e+09,0.000000e+00,0.000000e+00,1.638540e+10,NaN,NaN
3,0.000000e+00,2.789663e+09,0.000000e+00,2.891880e+10,0.000000e+00,0.000000e+00,760,760
4,5.037025e+10,3.346675e+09,1.104000e+08,4.921637e+10,9.976421e+09,1.492691e+10,437,385


# shape

In [55]:
train.shape # 301개의 instance와 10개의 피처가 있음. 

(301, 9)

In [56]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OC            301 non-null    object 
 1   revenue1      293 non-null    float64
 2   profit1       293 non-null    float64
 3   receivableS1  293 non-null    float64
 4   debt1         293 non-null    float64
 5   longLoan1     293 non-null    float64
 6   surplus1      293 non-null    float64
 7   employee1     291 non-null    float64
 8   employee2     288 non-null    float64
dtypes: float64(8), object(1)
memory usage: 21.3+ KB


In [57]:
train['OC'].value_counts() # 불균형 데이터

open      286
 close     15
Name: OC, dtype: int64

In [58]:
train.describe()

,revenue1,profit1,receivableS1,debt1,longLoan1,surplus1,employee1,employee2
count,2.930000e+02,2.930000e+02,2.930000e+02,2.930000e+02,2.930000e+02,2.930000e+02,291.000000,288.000000
mean,1.288175e+10,2.101700e+08,6.975862e+08,8.765310e+09,2.646564e+09,1.265935e+09,142.546392,134.326389
std,2.043543e+10,1.134673e+09,1.455332e+09,1.286112e+10,4.697523e+09,8.248321e+09,160.191073,151.061786
min,0.000000e+00,-4.696701e+09,0.000000e+00,0.000000e+00,0.000000e+00,-2.914970e+09,0.000000,0.000000
25%,3.252112e+09,-2.168279e+06,0.000000e+00,1.953077e+09,0.000000e+00,0.000000e+00,53.500000,53.750000
50%,5.524218e+09,8.130751e+07,2.158564e+08,4.457667e+09,1.136776e+09,0.000000e+00,80.000000,79.000000
75%,1.274839e+10,2.930329e+08,6.940605e+08,9.437917e+09,3.327098e+09,1.261600e+08,181.500000,170.000000
max,1.510000e+11,7.475427e+09,1.409257e+10,8.669725e+10,4.112490e+10,1.340000e+11,1200.000000,1200.000000


# 직원의 min값이 0인 것은 이상함
# revenue가 0 인것도 이상함
--> NAN 값인 것으로 추정되어 변환하겠다.

In [59]:
train.isna().sum()

OC               0
revenue1         8
profit1          8
receivableS1     8
debt1            8
longLoan1        8
surplus1         8
employee1       10
employee2       13
dtype: int64

# 결손 데이터를 대체한다. : inplace=True

In [61]:
for i in ['revenue1','profit1','receivableS1','debt1','longLoan1','surplus1','employee1','employee2']:
  train[i] = train[i].fillna(train[i].mean())

train.head()

,OC,revenue1,profit1,receivableS1,debt1,longLoan1,surplus1,employee1,employee2
0,open,4.217530e+09,2.251697e+08,4.700557e+08,6.828260e+08,3.510000e+08,1.496394e+09,62.0,64.0
1,open,1.288175e+10,2.101700e+08,6.975862e+08,8.765310e+09,2.646564e+09,1.265935e+09,801.0,813.0
2,open,1.004522e+09,4.186475e+07,8.095950e+06,9.241434e+07,0.000000e+00,0.000000e+00,234.0,1.0
3,open,7.250734e+10,9.028303e+08,0.000000e+00,4.236380e+10,7.253040e+09,8.643659e+09,663.0,663.0
4,open,4.904354e+10,8.074970e+07,4.099320e+09,4.989811e+10,1.350000e+10,9.025550e+09,206.0,197.0


In [62]:
train.isna().sum()

OC              0
revenue1        0
profit1         0
receivableS1    0
debt1           0
longLoan1       0
surplus1        0
employee1       0
employee2       0
dtype: int64

In [63]:
train.describe()

,revenue1,profit1,receivableS1,debt1,longLoan1,surplus1,employee1,employee2
count,3.010000e+02,3.010000e+02,3.010000e+02,3.010000e+02,3.010000e+02,3.010000e+02,301.000000,301.000000
mean,1.288175e+10,2.101700e+08,6.975862e+08,8.765310e+09,2.646564e+09,1.265935e+09,142.546392,134.326389
std,2.016112e+10,1.119442e+09,1.435796e+09,1.268848e+10,4.634466e+09,8.137600e+09,157.498594,147.752533
min,0.000000e+00,-4.696701e+09,0.000000e+00,0.000000e+00,0.000000e+00,-2.914970e+09,0.000000,0.000000
25%,3.279139e+09,0.000000e+00,7.203600e+04,2.029170e+09,0.000000e+00,0.000000e+00,54.000000,56.000000
50%,5.676235e+09,9.353360e+07,2.401758e+08,4.719856e+09,1.390000e+09,0.000000e+00,84.000000,80.000000
75%,1.288175e+10,2.882976e+08,6.975862e+08,9.265154e+09,3.242405e+09,2.209221e+08,177.000000,161.000000
max,1.510000e+11,7.475427e+09,1.409257e+10,8.669725e+10,4.112490e+10,1.340000e+11,1200.000000,1200.000000


In [64]:
train[train['employee1']==0].count() # 직원이 0명인 병원이 존재하나보다~

OC              1
revenue1        1
profit1         1
receivableS1    1
debt1           1
longLoan1       1
surplus1        1
employee1       1
employee2       1
dtype: int64

In [65]:
train[train['employee2']==0].count() # 직원이 0명인 병원이 존재하나보다~

OC              2
revenue1        2
profit1         2
receivableS1    2
debt1           2
longLoan1       2
surplus1        2
employee1       2
employee2       2
dtype: int64

In [66]:
y_train = train['OC']
X_train = train.iloc[:,1:]

X_train.head()

,revenue1,profit1,receivableS1,debt1,longLoan1,surplus1,employee1,employee2
0,4.217530e+09,2.251697e+08,4.700557e+08,6.828260e+08,3.510000e+08,1.496394e+09,62.0,64.0
1,1.288175e+10,2.101700e+08,6.975862e+08,8.765310e+09,2.646564e+09,1.265935e+09,801.0,813.0
2,1.004522e+09,4.186475e+07,8.095950e+06,9.241434e+07,0.000000e+00,0.000000e+00,234.0,1.0
3,7.250734e+10,9.028303e+08,0.000000e+00,4.236380e+10,7.253040e+09,8.643659e+09,663.0,663.0
4,4.904354e+10,8.074970e+07,4.099320e+09,4.989811e+10,1.350000e+10,9.025550e+09,206.0,197.0


In [67]:
y_train.head()

0    open
1    open
2    open
3    open
4    open
Name: OC, dtype: object

# 테스트 데이터 결측치 처리

In [68]:
test.head()

,revenue1,profit1,receivableS1,debt1,longLoan1,surplus1,employee1,employee2
0,6.682486e+10,5.036692e+08,7.242580e+08,6.769090e+10,6.319097e+09,8.944587e+09,693,693
1,3.495758e+10,4.525557e+08,6.610290e+08,6.626737e+10,1.720000e+10,1.046277e+10,379,371
2,2.326031e+10,-2.148749e+09,7.337107e+09,0.000000e+00,0.000000e+00,1.638540e+10,NaN,NaN
3,0.000000e+00,2.789663e+09,0.000000e+00,2.891880e+10,0.000000e+00,0.000000e+00,760,760
4,5.037025e+10,3.346675e+09,1.104000e+08,4.921637e+10,9.976421e+09,1.492691e+10,437,385


In [71]:
test.isna().sum()

revenue1         0
profit1          0
receivableS1     0
debt1            0
longLoan1        0
surplus1         0
employee1        8
employee2       15
dtype: int64

In [73]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   revenue1      127 non-null    float64
 1   profit1       127 non-null    float64
 2   receivableS1  127 non-null    float64
 3   debt1         127 non-null    float64
 4   longLoan1     127 non-null    float64
 5   surplus1      127 non-null    float64
 6   employee1     119 non-null    object 
 7   employee2     112 non-null    object 
dtypes: float64(6), object(2)
memory usage: 8.1+ KB


# 1000 이상의 숫자에 test는 쉼표가 있음.

In [74]:
test.employee1 = test.employee1.astype('str').str.replace(",", "").astype('float')
test.employee2 = test.employee2.astype('str').str.replace(",", "").astype('float')

In [75]:
for i in ['employee1','employee2']:
  test[i] = test[i].fillna(test[i].mean())

test.head()

,revenue1,profit1,receivableS1,debt1,longLoan1,surplus1,employee1,employee2
0,6.682486e+10,5.036692e+08,7.242580e+08,6.769090e+10,6.319097e+09,8.944587e+09,693.000000,693.000000
1,3.495758e+10,4.525557e+08,6.610290e+08,6.626737e+10,1.720000e+10,1.046277e+10,379.000000,371.000000
2,2.326031e+10,-2.148749e+09,7.337107e+09,0.000000e+00,0.000000e+00,1.638540e+10,204.252101,189.223214
3,0.000000e+00,2.789663e+09,0.000000e+00,2.891880e+10,0.000000e+00,0.000000e+00,760.000000,760.000000
4,5.037025e+10,3.346675e+09,1.104000e+08,4.921637e+10,9.976421e+09,1.492691e+10,437.000000,385.000000
